<a href="https://colab.research.google.com/github/ndeleonardis/DataSetNoeliaDeLeonardis/blob/main/entregable_datasciense_3_noelia_de_leonardis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción


Para este trabajo se usará un **dataset confidencial** vinculado a **Fondos Comunes de Inversión del banco de Córdoba**, llamados **Fondos Champaquí**.

Antes que nada vamos a definir que es un **Fondo Común de Inversión**, fondo común de inversión es un vehículo financiero que agrupa el capital de múltiples inversores para invertir en una cartera diversificada de activos, como acciones, bonos o bienes raíces. La gestión de estas inversiones es realizada por profesionales especializados, y los rendimientos se distribuyen entre los inversores en función de su participación en el fondo.

A modo general, se trabajo en la obtención de información, la misma esta parcializada ya que solo se tiene acceso a la base de fondos y no a otras integraciones.

Para dar un pantallazo, para que un cliente pueda operar en Fondos Champaquí, debe abrir una cuenta de inversión, la misma queda registrada en otra base de datos a la cual no tenemos acceso, pero los datos del cliente se preplican de forma parcial.


Cada cliente para poder operar como indicamos tiene que poseer una cuenta de inversión y realizar el perfil de inversor, que dice que adversión al riesgo puede tener la persona, conservadora, moderada, o agresiva. Basicamente es que nivel de riesgo esta dispuesta a tomar para obtener ganancias.

El trabajo va estar dividido en 2 partes primordiales que serán la solicitudes de suscripción y rescate cursadas en el año 2023.

# Hipótesis que trataremos de responder haciendo uso del data wrangling  y EDA

**Patrones de Suscripción y Rescate según el Perfil de Riesgo**

**Hipótesis:** Los inversores con un perfil de riesgo más alto realizan más solicitudes de suscripción y rescates en comparación con los inversores con perfiles más conservadores.\

**Los hombres invierten más que las mujeres**

**Hipótesis:** Es verdad que los hombres invierten más que las mujeres?.

**En que temporada se concentran más suscripciones y rescates?**

**Hipótesis:** Hay patrones estacionales en las solicitudes de suscripción y rescates, como un aumento durante ciertos meses del año.


In [15]:
#Importación de Librerias
import pandas as pd
import numpy as np

In [4]:

# Cargamaos el archivo usamos URL cruda del archivo CSV que esta en nuestro GitHub
url = 'https://raw.githubusercontent.com/ndeleonardis/DataSetNoeliaDeLeonardis/main/dataset_suscripciones_2023.csv'


# Cargar el archivo CSV en un DataFrame de Pandas
try:
    df = pd.read_csv(url)
    print("El archivo CSV se ha cargado exitosamente.")
except Exception as e:
    print("Se produjo un error al cargar el archivo CSV:", e)

El archivo CSV se ha cargado exitosamente.


In [6]:
#Exploramos el dataframe con el comando info que nos da información del tipo de datos y nro de entradas y valores nulos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224518 entries, 0 to 224517
Data columns (total 43 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CodFondo              224518 non-null  int64  
 1   Nombre                224518 non-null  object 
 2   CodAgColocador        224518 non-null  int64  
 3   CodSucursal           224518 non-null  int64  
 4   CodSolSusc            224518 non-null  int64  
 5   CodTpValorCp          224518 non-null  int64  
 6   CodCanalVta           1404 non-null    float64
 7   CodOficialCta         224518 non-null  int64  
 8   CodCondicionIngEgr    224518 non-null  int64  
 9   CodMoneda             224518 non-null  int64  
 10  CodCuotapartista      224518 non-null  int64  
 11  NumSolicitud          224518 non-null  int64  
 12  Importe               224518 non-null  float64
 13  FechaConcertacion     224518 non-null  object 
 14  FechaAcreditacion     224518 non-null  object 
 15  

Luego de ejecutar este comando vemos que:
**Tenemos un total de 224.518 registros** (RangeIndex: 224518 entries, 0 to 224517)
Con la salida de este comando vemos las siguientes observaciones:

* **Campos Fechas (FechaConcertacion / FechaAcreditacion / FechaIngreso)** no tienen el formato fecha
* **CodCanalVta** solo tiene 1404 valores No nulls
* **Observaciones** que nos sera de utilidad tiene  222.413 non-null es decir que nos faltan 2.105 con datos, ya vamos a filtrar por esas columnas a fin de determinar que realizaremos.
También vemos que las fechas no estan en formato de fecha.
* Datos con valores todos nullos (Perfil/PerfilRiesgoFechaVto  )


In [14]:
df_filtered = df[df['CodCanalVta'].notna()]
print(df_filtered)



        CodFondo                  Nombre  CodAgColocador  CodSucursal  \
900            1  Champaquí Ahorro Pesos               1            4   
901            1  Champaquí Ahorro Pesos               1            4   
927            1  Champaquí Ahorro Pesos               1            4   
928            1  Champaquí Ahorro Pesos               1            4   
929            1  Champaquí Ahorro Pesos               1            4   
...          ...                     ...             ...          ...   
224507         9  Champaqui Renta Futura               1          152   
224511         9  Champaqui Renta Futura               1          152   
224512         9  Champaqui Renta Futura               1          152   
224514         9  Champaqui Renta Futura               1          161   
224516         9  Champaqui Renta Futura               1          161   

        CodSolSusc  CodTpValorCp  CodCanalVta  CodOficialCta  \
900         164095             4          2.0              

Como no es un dato que vamos a ser foco lo dejaremos.

In [ ]:
#Analizamos el campo observaciones, que es de interes porque es una forma que el sistema mapeo con que CUIT o CUIL se realizó la transacción.
# Vemos que el mismo tiene un formato "Nro|AR1B-AR1A" donde AR1B signnifica CUIL y esta en general asociado a personas físicas, mientras que AR1A "27324839386|AR1B"
# Tenemos un campo "EsPersonaFisica" que nos indica si las persona es física (-1) o jurídica  (0)
# Como los clientes son mayores a 18 años, hasta esa fecha (año 2021) los CUIL se asignaban por afip 20 para masculino y 27 para femenino.
#Entonces trataremos de asignar sexo por ahi.

In [25]:
# Filtrar los registros donde 'EsPersonaFisica' es -1
df_filtered = df[df['EsPersonaFisica'] == -1]
print("Cantidad de registros de personas físicas:", df_filtered.shape[0])
print()
# Filtrar los registros donde 'EsPersonaFisica' es -1 y 'Observaciones' no es nulo
df_filtered = df[(df['EsPersonaFisica'] == -1) & (df['Observaciones'].notna())]
print("Cantidad de registros de personas físicas y que en el campo observaciones no sea nulo:", df_filtered.shape[0])

# Crear una nueva columna 'Sexo' e inicializarla con valores NaN
df_filtered['Sexo'] = pd.NA

# Asignar valores a la columna 'Sexo' basados en el campo 'Observaciones'
df_filtered.loc[df_filtered['Observaciones'].str.startswith('20', na=False), 'Sexo'] = 1
df_filtered.loc[df_filtered['Observaciones'].str.startswith('27', na=False), 'Sexo'] = 0

# Crear una nueva columna 'Sexo' e inicializarla con valores NaN
df_filtered['Sexo'] = pd.NA

# Asignar valores a la columna 'Sexo' basados en el campo 'Observaciones'
df_filtered.loc[df_filtered['Observaciones'].str.startswith('20', na=False), 'Sexo'] = 1
df_filtered.loc[df_filtered['Observaciones'].str.startswith('27', na=False), 'Sexo'] = 0


Cantidad de registros de personas físicas: 171938

Cantidad de registros de personas físicas y que en el campo observaciones no sea nulo: 171099


In [28]:


# Supongamos que tu DataFrame se llama df

# Filtrar los registros donde 'EsPersonaFisica' es -1
df_filtered = df[df['EsPersonaFisica'] == -1].copy()
print("Cantidad de registros de personas físicas:", df_filtered.shape[0])
print()

# Filtrar los registros donde 'EsPersonaFisica' es -1 y 'Observaciones' no es nulo
df_filtered = df_filtered[df_filtered['Observaciones'].notna()]
print("Cantidad de registros de personas físicas y que en el campo 'Observaciones' no sea nulo:", df_filtered.shape[0])
print()

# Crear una nueva columna 'Sexo' e inicializarla con valores NaN
df_filtered['Sexo'] = pd.NA

# Asignar valores a la columna 'Sexo' basados en el campo 'Observaciones'
df_filtered.loc[df_filtered['Observaciones'].str.startswith('20', na=False), 'Sexo'] = 1
df_filtered.loc[df_filtered['Observaciones'].str.startswith('27', na=False), 'Sexo'] = 0

# Contar la cantidad de cada valor en la columna 'Sexo'
sexo_counts = df_filtered['Sexo'].value_counts(dropna=False)
print("Cantidad de valores en la columna 'Sexo':")
print(sexo_counts)


Cantidad de registros de personas físicas: 171938

Cantidad de registros de personas físicas y que en el campo 'Observaciones' no sea nulo: 171099

Cantidad de valores en la columna 'Sexo':
Sexo
1       78163
0       75962
<NA>    16974
Name: count, dtype: int64


In [17]:
df_filtered.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 171938 entries, 0 to 224482
Data columns (total 44 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CodFondo              171938 non-null  int64  
 1   Nombre                171938 non-null  object 
 2   CodAgColocador        171938 non-null  int64  
 3   CodSucursal           171938 non-null  int64  
 4   CodSolSusc            171938 non-null  int64  
 5   CodTpValorCp          171938 non-null  int64  
 6   CodCanalVta           410 non-null     float64
 7   CodOficialCta         171938 non-null  int64  
 8   CodCondicionIngEgr    171938 non-null  int64  
 9   CodMoneda             171938 non-null  int64  
 10  CodCuotapartista      171938 non-null  int64  
 11  NumSolicitud          171938 non-null  int64  
 12  Importe               171938 non-null  float64
 13  FechaConcertacion     171938 non-null  object 
 14  FechaAcreditacion     171938 non-null  object 
 15  Obser